## Time to apply to Video! </br>
the next step would be to analyze these for the whole video. </br>
So keep the background image, load every new frame of the video, run your calculations, and then save it to a video file. This tutorial might be helpful.

In [1]:
# example codes
# reference : https://learnopencv.com/reading-and-writing-videos-using-opencv/
import cv2 
 
# Create a video capture object, in this case we are reading the video from a file
vid_capture = cv2.VideoCapture('Resources/Cars.mp4')
 
if (vid_capture.isOpened() == False):
  print("Error opening the video file")
# Read fps and frame count
else:
  # Get frame rate information
  # You can replace 5 with CAP_PROP_FPS as well, they are enumerations
  fps = vid_capture.get(5)
  print('Frames per second : ', fps,'FPS')
 
  # Get frame count
  # You can replace 7 with CAP_PROP_FRAME_COUNT as well, they are enumerations
  frame_count = vid_capture.get(7)
  print('Frame count : ', frame_count)
 
while(vid_capture.isOpened()):
  # vid_capture.read() methods returns a tuple, first element is a bool 
  # and the second is frame
  ret, frame = vid_capture.read()
  if ret == True:
    cv2.imshow('Frame',frame)
    # 20 is in milliseconds, try to increase the value, say 50 and observe
    key = cv2.waitKey(20)
     
    if key == ord('q'):
      break
  else:
    break
 
# Release the video capture object
vid_capture.release()
cv2.destroyAllWindows()

Error opening the video file


OpenCV: Couldn't read video stream from file "Resources/Cars.mp4"


In [ ]:
import os

color_filenames = os.listdir("color")
depth_filenames = os.listdir("depth")

background_depth = loadImage(depth_filenames[0])

result_vid = cv.VideoWriter 

prev_bounding_box = ...

for i in range(depth_filenames):

    load Image

    take depth difference

    process bounding box

    
 # calculating the bounding box i/u

    result_vid.write(image)

result_vid.release()